# The diffusion equation

Or, hillslope evolution as we knew it in the 1960s.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import firedrake
from firedrake import Constant, exp, inner, grad, dx
import irksome
from irksome import Dt

## Solving steady problems

Firedrake includes functions to create a few standard meshes.
Later we can see how to load in a geometry that you create with a mesh generator.

In [ ]:
mesh = ...
radius = 200.0
mesh.coordinates.dat.data[:] *= radius

Make a plot of the mesh.
We can visualize the numeric IDs of the different boundary segments by adding a legend.
Getting the wrong boundary conditions is one of the most common errors!

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(mesh, axes=ax)
ax.legend(loc="upper right");

Create a finite element.
This object describes how we're going to discretize the problem.
Here we'll just use continuous Galerkin elements of degree 1, so the solution is approximated as being piecewise linear inside each triangle.

In [ ]:
element = ...

A finite element just describes how to represent a function in general.
It does not have any connection to the mesh as such.
A function space is created from a mesh and an element.

In [ ]:
V = ...

First, we'll get an object out of the mesh that describes points within the domain.
We can then manipulate this object in order to create complex algebraic expressions.
Here we'll make a function that represents an uplift rate consisting of two Gaussian peaks.

In [ ]:
x = ...
R = Constant(radius)

u_0 = Constant(7.5e-4)

expr = ...

The expression that we created is purely symbolic.
In order to get out an object with a ball of data underneath that we can, say, make a plot of, or use in a PDE, we need to interpolate that algebraic expression into a function space.

In [ ]:
U = ...

Firedrake includes some plotting functions that call out to matplotlib.
Here we'll make a 3D plot of the uplift.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
firedrake.trisurf(U, axes=ax);

Now we'll create a variable that will hold the solution to the PDE.

In [ ]:
z = ...

In order to create a symbolic representation of the PDE, we first need to make a test function.

In [ ]:
w = ...

Now we'll create the variational form of the steady state problem.
The mathematical problem we wish to solve is to find the elevation $z$ such that, for all test functions $w$,
$$\int_\Omega \left(k\nabla z\cdot\nabla w - U\;w\right)dx = 0.$$

In [ ]:
k = Constant(0.0035)  # m² / yr
F = ...

But we also need to supply a boundary condition.
Here we'll use the Dirichlet condition
$$z|_{\partial\Omega} = 0.$$

In [ ]:
bc = ...

Almost there!
To solve the PDE, we first create an object describing what problem we wish to solve.
Then we create an object storing the information that we need in order to solve this problem.

In [ ]:
problem = ...
solver = ...

Finally, we can invoke a method to solve the PDE.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
firedrake.trisurf(z, axes=ax);

If we alter the right-hand side of the PDE and invoke the solve method again, we get a different solution.

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
firedrake.trisurf(z, axes=ax);

## Time-dependent problems

What if we instead wanted to solve a time-dependent problem?
The variational form is:
$$\int_\Omega\left(\partial_t z\; w + k\nabla z\cdot\nabla w - U\;w\right)dx = 0.$$

In principle, we could also do time-dependent problems using what I've shown above.
First, we create two functions for the values of the elevation at the previous and the current timestep $z_{n - 1}$ and $z_n$.
We then pick a timestep $\delta t$ and create the variational form

$$\int_\Omega\left(\frac{z_n - z_{n - 1}}{\delta t}w + k\nabla z_n\cdot\nabla w - Uw\right)dx = 0.$$

We then create boundary conditions, a problem object, and a solver object.
Finally, we create a big loop.
In each step, we call `.solve` to obtain the next elevation value and then assign the newly obtained value to the function holding the previous value.

This all works OK but as soon as we want to use a more sophisticated timestepping scheme we're hosed.
Instead, we can code up the time-dependent problem as such.

In [ ]:
F = ...

The Irksome package does **I**mplicit **R**unge-**K**utta methods.
Here we're asking to use the backward Euler discretization.

In [ ]:
method = ...

We need to decide how long to integrate for.
We can guess at a reasonable timescale based on the uplift rate and the radius of the domain.

In [ ]:
final_time = 3 * radius / float(u_0)
print(final_time)

Next we need to choose a timestep.
There are principled ways to do this.
But let's not kid, we always use trial and error.

In [ ]:
timestep = final_time / 400.0
dt = Constant(timestep)

Finally we create a time stepper object.
This is analogous to the solver that we used before for steady problems.

In [ ]:
stepper = ...

And now a big loop.

In [ ]:
num_steps = int(final_time / timestep)

zs = []
for step in trange(num_steps):
    ...

And a movie.

In [ ]:
%%capture

fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(zs[0], axes=ax, vmin=0, vmax=200.0, num_sample_points=4)
fig.colorbar(colors)

In [ ]:
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(z):
    colors.set_array(fn_plotter(z))

animation = FuncAnimation(fig, animate, tqdm(zs), interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())